In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
classes = []
classes = next(os.walk('hw5_data/train/'))[1]
class_num = len(classes)
#print(classes)

In [3]:
def DataLoader(dir_path):
    data = []
    label = []
    for c in classes:
        #num = 0
        class_path = os.path.join(dir_path, c)
        fname = os.listdir(class_path)
        for n in fname:
            img_path = os.path.join(class_path, n)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None: 
                img = cv2.resize(img, (256, 256))
                # normalize
                img = img.astype(np.float32)
                img -= np.mean(img)
                img /= np.linalg.norm(img)
                data.append(img)
                label.append(c)
                #num += 1
        #print(f"{c}: {num}")
    return np.array(label), np.array(data)

train_label , train_img = DataLoader("hw5_data/train/")
test_label , test_img = DataLoader("hw5_data/test/")

In [4]:
#Dense SIFT and K-means
from cyvlfeat.kmeans import kmeans
from cyvlfeat.sift.dsift import dsift

def get_cluster(images, cluster_size, step_size=10):
    bag_of_features = []
    for img in images:
        _, descriptors = dsift(img, step=step_size, fast=True)
        if descriptors is not None:
            for des in descriptors:
                bag_of_features.append(des)
    bag_of_features = np.array(bag_of_features).astype('float64')
    clusters = kmeans(bag_of_features, cluster_size)
    return clusters

In [5]:
#Get histogram
from scipy.cluster.vq import vq

def get_hist(img, code_book, step_size=10):
    _, descriptors = dsift(img, step=step_size, fast=True)
    descriptors = descriptors.astype('float64')
    code, dist = vq(descriptors,code_book)
    hist, bin_edges = np.histogram(code, bins=range(len(code_book)+1))
    return hist

In [6]:
#SVM
from libsvm.svmutil import *

In [32]:
train_label = []
test_label = []
for i in range(15):
    for _ in range(100):
        train_label.append(i)
    for _ in range(10):
        test_label.append(i)
train_hist = []

code_book = get_cluster(train_img, 300, step_size=5)
for img in train_img:
    train_hist.append(get_hist(img, code_book, step_size=5))
train_hist = np.array(train_hist)
test_hist = []
for img in test_img:
    test_hist.append(get_hist(img, code_book, step_size=5))
test_hist = np.array(test_hist)
#param = svm_parameter()
#-s 0:random selection
#-q: default calculation
#-v 5: 5-fold validation
#-t 0: kernel type linear
best_acc = 0

for c in range(-5,10,1):
    for g in range (1, 20, 3):
        params = f'-q -v 5 -s 0 -t 0 -c {2**c} -g {0.001*g}'
        CV_ACC = svm_train(train_label,train_hist, params)
        if CV_ACC > best_acc:
            best_acc = CV_ACC
            best_params = params
        print(f'{params}')
print("----------------------")
print(best_acc)
print(best_params)

Cross Validation Accuracy = 64.4%
-q -v 5 -s 0 -t 0 -c 0.03125 -g 0.001
Cross Validation Accuracy = 63.8667%
-q -v 5 -s 0 -t 0 -c 0.03125 -g 0.004
Cross Validation Accuracy = 66%
-q -v 5 -s 0 -t 0 -c 0.03125 -g 0.007
Cross Validation Accuracy = 66.1333%
-q -v 5 -s 0 -t 0 -c 0.03125 -g 0.01
Cross Validation Accuracy = 65.8%
-q -v 5 -s 0 -t 0 -c 0.03125 -g 0.013000000000000001
Cross Validation Accuracy = 65.8667%
-q -v 5 -s 0 -t 0 -c 0.03125 -g 0.016
Cross Validation Accuracy = 66.7333%
-q -v 5 -s 0 -t 0 -c 0.03125 -g 0.019
Cross Validation Accuracy = 66.2%
-q -v 5 -s 0 -t 0 -c 0.0625 -g 0.001
Cross Validation Accuracy = 65.8%
-q -v 5 -s 0 -t 0 -c 0.0625 -g 0.004
Cross Validation Accuracy = 65.8%
-q -v 5 -s 0 -t 0 -c 0.0625 -g 0.007
Cross Validation Accuracy = 67.3333%
-q -v 5 -s 0 -t 0 -c 0.0625 -g 0.01
Cross Validation Accuracy = 66.4667%
-q -v 5 -s 0 -t 0 -c 0.0625 -g 0.013000000000000001
Cross Validation Accuracy = 66.8%
-q -v 5 -s 0 -t 0 -c 0.0625 -g 0.016
Cross Validation Accuracy 

In [33]:
params = f'-q -s 0 -t 0 -c 4 -g 0.013'
m = svm_train(train_label, train_hist, params)
pred, acc, _ = svm_predict(test_label,test_hist, m)

Accuracy = 69.3333% (104/150) (classification)
